<a href="https://colab.research.google.com/github/wesleydinizdatascientist/ai-lab/blob/wesleydinizdatascientist-mba_fiap_credit_profiles/mba_fiap_clustering_credit_profiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns


from sklearn.compose    import ColumnTransformer
from sklearn.pipeline     import Pipeline
from sklearn.cluster      import KMeans
import joblib

In [ ]:
del df

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
path = '/content/Base_ScoreCredito_QuantumFinance.csv'
df   = pd.read_csv(path, sep=';')

# Detalhamento

In [4]:
df

,id,idade,sexo,estado_civil,escola,Qte_dependentes,tempo_ultimoservico,trabalha,vl_salario_mil,reg_moradia,casa_propria,vl_imovel_em_mil,Qte_cartoes,Qte_carros,SCORE_CREDITO
0,708082083,45,F,casado,graduacao,3,40,1,"40,089665024322",3,0,0,1,1,778
1,708083283,58,M,solteiro,ensino fundam,0,44,1,"66,557645274838",3,0,0,1,0,"276,3"
2,708084558,46,M,divorciado,doutorado,3,35,1,"123,68182089138",6,0,0,1,1,401
3,708085458,34,F,solteiro,mestrado,0,22,1,"19,7159343168329",6,0,0,1,0,"347,4"
4,708086958,49,F,casado,mestrado,2,36,0,0,3,1,437,1,2,476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,828288333,50,M,casado,mestrado,0,45,1,"100,128970755",4,0,0,1,0,470
10123,828291858,38,M,solteiro,ensino fundam,0,31,1,"89,80599574656",6,0,0,1,1,432
10124,828294933,49,F,casado,graduacao,3,43,1,"19,7993057098452",3,1,252,1,1,406
10125,828298908,54,M,casado,ensino medio,4,48,1,"194,78317642784",4,0,0,1,1,387


In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df[['vl_salario_mil',
              'vl_imovel_em_mil',
              'tempo_ultimoservico',
              'Qte_cartoes',
              'Qte_carros',
              'idade'
              ]].describe()

,vl_imovel_em_mil,tempo_ultimoservico,Qte_cartoes,Qte_carros,idade
count,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,208.999111,34.133208,1.083638,0.623383,46.325960
std,378.495229,8.221428,0.333784,0.569796,8.016814
min,0.000000,7.000000,1.000000,0.000000,26.000000
25%,0.000000,30.000000,1.000000,0.000000,41.000000
50%,0.000000,34.000000,1.000000,1.000000,46.000000
75%,289.000000,39.000000,1.000000,1.000000,52.000000
max,1800.000000,57.000000,4.000000,2.000000,73.000000


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# 1. Gerar um conjunto de pontos (200 amostras, 3 centros)
X, _ = make_blobs(n_samples=200, centers=3, n_features=2, random_state=42)

# 2. Instanciar e ajustar o KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X)

# 3. Visualizar o resultado
plt.figure(figsize=(6, 4))
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
            marker='X', s=200, edgecolor='k')
plt.title("Exemplo de K-Means em Dados Sem Rótulos")
plt.xlabel("Característica 1")
plt.ylabel("Característica 2")
plt.show()


# Limpeza de dados (Data Cleaning)

In [ ]:
id_col = df['id']
df_cleaned = df.drop(columns=['id', 'SCORE_CREDITO'])


sexo_convert = {'F': 'mulher', 'M': 'homem'}
df_cleaned['sexo'] = df_cleaned['sexo'].map(sexo_convert)


escola_convert = {'ensino fundam': 'ensino_fundamental',
                  'ensino medio': 'ensino_medio',
                  'graduacao': 'graduacao',
                  'mestrado': 'mestrado',
                  'doutorado': 'doutorado'}

df_cleaned['escola'] = df_cleaned['escola'].map(escola_convert)


df_cleaned['vl_salario_mil'] = df_cleaned['vl_salario_mil'].str.replace(',', '.').astype(float)

df_cleaned['vl_imovel_em_mil'] = df_cleaned['vl_imovel_em_mil'].astype(float)

In [ ]:
df_cleaned.describe()

# Engenharia de variáveis (Feature Engineering)

In [ ]:
df_preproced = pd.DataFrame(df_cleaned)

In [ ]:
df_preproced['escola'] = df_preproced['escola'].replace({
    'ensino_fundamental': 'ensino_medio',
    'ensino_medio': 'ensino_medio',
    'graduacao': 'ensino_superior',
    'mestrado': 'ensino_superior',
    'doutorado': 'ensino_superior'
})

def resumindo_estado_civil(x):
    if x in ['casado', 'solteiro']:
        return x
    else:
        return 'sem_relacionamento_formal'

df_preproced['estado_civil'] = df_preproced['estado_civil'].apply(resumindo_estado_civil)

mapa_regiao = {
    1: 'zona_A',
    2: 'zona_B',
    3: 'zona_C',
    4: 'zona_D',
    5: 'zona_E',
    6: 'zona_F'
}

df_preproced['regiao_moradia'] = df_preproced['reg_moradia'].map(mapa_regiao)


alta_renda = ['zona_A', 'zona_B']
media_renda = ['zona_C', 'zona_F']
baixa_renda = ['zona_D', 'zona_E']

def segmentando_regiao(x):
    if x in alta_renda:
        return 'regiao_classe_A'
    elif x in media_renda:
        return 'regiao_classe_B'
    else:
        return 'regiao_classe_C'

df_preproced['grupo_regiao'] = df_preproced['regiao_moradia'].apply(segmentando_regiao)

df_preproced['renda_media_por_dependente'] = df_preproced['vl_salario_mil'] / (df_preproced['Qte_dependentes'] + 1)


df_preproced['log_renda_media_por_dependente'] = np.log1p(df_preproced['renda_media_por_dependente'])

df_preproced = pd.get_dummies(df_preproced,
                              columns=['sexo',
                                       'estado_civil',
                                       'escola',
                                       'regiao_moradia',
                                       'grupo_regiao'],
                              drop_first=True)

dummy_cols = df_preproced.filter(
    regex='^sexo_|^estado_civil_|^escola_|^regiao_moradia_|^grupo_regiao_'
).columns

df_preproced[dummy_cols] = df_preproced[dummy_cols].astype(int)

In [ ]:
df_preproced.describe()

,idade,Qte_dependentes,tempo_ultimoservico,trabalha,vl_salario_mil,reg_moradia,casa_propria,vl_imovel_em_mil,Qte_cartoes,Qte_carros,renda_media_por_dependente,log_renda_media_por_dependente,sexo_mulher,estado_civil_sem_relacionamento_formal,estado_civil_solteiro,escola_ensino_superior,regiao_moradia_zona_B,regiao_moradia_zona_C,regiao_moradia_zona_D,regiao_moradia_zona_E,regiao_moradia_zona_F,grupo_regiao_regiao_classe_B,grupo_regiao_regiao_classe_C
count,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.00000,10127.000000
mean,46.325960,1.547447,34.133208,0.911919,70.209002,3.783450,0.352424,208.999111,1.083638,0.623383,39.244960,3.060145,0.529081,0.147823,0.389355,0.443567,0.114940,0.221981,0.189888,0.185247,0.182779,0.40476,0.375136
std,8.016814,1.489079,8.221428,0.283427,55.567032,1.580486,0.477749,378.495229,0.333784,0.569796,42.343235,1.316731,0.499178,0.354942,0.487628,0.496830,0.318966,0.415599,0.392232,0.388517,0.386504,0.49087,0.484182
min,26.000000,0.000000,7.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,41.000000,0.000000,30.000000,1.000000,21.330536,3.000000,0.000000,0.000000,1.000000,0.000000,9.567054,2.357741,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,46.000000,1.000000,34.000000,1.000000,66.486331,4.000000,0.000000,0.000000,1.000000,1.000000,23.501650,3.198740,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,52.000000,3.000000,39.000000,1.000000,107.544225,5.000000,1.000000,289.000000,1.000000,1.000000,55.621694,4.036392,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000
max,73.000000,5.000000,57.000000,1.000000,233.301793,6.000000,1.000000,1800.000000,4.000000,2.000000,230.130628,5.442983,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [ ]:
df_preproced['tempo_ultimoservico'] = np.log1p(df_preproced['tempo_ultimoservico'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
df_preproced['renda_zscore'] = scaler.fit_transform(df_cleaned[['vl_salario_mil']])
df_preproced['renda_media_por_dependente_zscore'] = scaler.fit_transform(df_preproced[['renda_media_por_dependente']])

In [ ]:
df_preproced['log_vl_imovel_em_mil'] = np.log1p(df_preproced['vl_imovel_em_mil'])

df_preproced['log_Qte_carros'] = np.log1p(df_preproced['Qte_carros'])

df_preproced['log_Qte_cartoes'] = np.log1p(df_preproced['Qte_cartoes'])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
X[numeric_cols].boxplot()
plt.xticks(rotation=45)
plt.title("Boxplot das variáveis já escaladas")
plt.tight_layout()
plt.show()


In [ ]:
# Salva em CSV, sem a coluna de índices
df_preproced.to_csv('dataset_kmeans.csv', index=False, encoding='utf-8')

# Fazendo uma análise descritiva da base

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ajustes de estilo gerais
sns.set(style="whitegrid")

# Cria figura com 3 linhas e 3 colunas
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(24, 18))
fig.suptitle("Dashboard Analítico Unificado", fontsize=20, fontweight='bold', y=0.98)

# 1) Histograma de idade por sexo
sns.histplot(
    data=df_preproced,
    x='idade',
    hue='sexo',
    multiple='stack',
    palette='Set2',
    bins=20,
    ax=axs[0, 0]
)
axs[0, 0].set_title("Distribuição de Idade por Sexo")
axs[0, 0].set_xlabel("Idade")
axs[0, 0].set_ylabel("Qtd Registros")
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.3)

# 2) Boxplot de idade por sexo
sns.boxplot(
    data=df_preproced,
    x='sexo',
    y='idade',
    palette='pastel',
    ax=axs[0, 1]
)
axs[0, 1].set_title("Boxplot da Idade por Sexo")
axs[0, 1].set_xlabel("Sexo")
axs[0, 1].set_ylabel("Idade")
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.3)

# 3) Violin plot: idade por escolaridade e sexo
sns.violinplot(
    data=df_preproced,
    x='escola',
    y='idade',
    hue='sexo',
    split=True,
    palette='Set2',
    order=['ensino_medio', 'ensino_superior'],
    ax=axs[0, 2]
)
axs[0, 2].set_title("Idade por Escolaridade e Sexo")
axs[0, 2].set_xlabel("Escolaridade")
axs[0, 2].set_ylabel("Idade")
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.3)

# 4) Violin plot: renda por escolaridade e sexo
sns.violinplot(
    data=df_preproced,
    x='escola',
    y='vl_salario_mil',
    hue='sexo',
    split=True,
    palette='Set2',
    order=['ensino_medio', 'ensino_superior'],
    ax=axs[1, 0]
)
axs[1, 0].set_title("Renda (mil R$) por Escolaridade e Sexo")
axs[1, 0].set_xlabel("Escolaridade")
axs[1, 0].set_ylabel("Salário (mil R$)")
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.3)

# 5) Boxplot: salário por região de moradia
sns.boxplot(
    data=df_preproced,
    x='regiao_moradia',
    y='vl_salario_mil',
    palette='Set2',
    order=['zona_A','zona_B','zona_C','zona_D','zona_E','zona_F'],
    ax=axs[1, 1]
)
axs[1, 1].set_title("Salário por Região de Moradia")
axs[1, 1].set_xlabel("Região")
axs[1, 1].set_ylabel("Salário (mil R$)")
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.3)

# 6) Boxplot: valor do imóvel por região de moradia
sns.boxplot(
    data=df_preproced,
    x='regiao_moradia',
    y='vl_imovel_em_mil',
    palette='Set2',
    order=['zona_A','zona_B','zona_C','zona_D','zona_E','zona_F'],
    ax=axs[1, 2]
)
axs[1, 2].set_title("Valor do Imóvel por Região")
axs[1, 2].set_xlabel("Região")
axs[1, 2].set_ylabel("Valor Imóvel (mil R$)")
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.3)

# 7) Boxplot: salário por grupo de moradia
sns.boxplot(
    data=df_preproced,
    x='grupo_regiao',
    y='vl_salario_mil',
    palette='Set2',
    order=['regiao_classe_A','regiao_classe_B','regiao_classe_C'],
    ax=axs[2, 0]
)
axs[2, 0].set_title("Salário por Classe de Moradia")
axs[2, 0].set_xlabel("Classe")
axs[2, 0].set_ylabel("Salário (mil R$)")
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.3)

# 8) Boxplot: renda média por dependente (com mediana anotada)
sns.boxplot(
    data=df_preproced,
    x='grupo_regiao',
    y='renda_media_por_dependente',
    palette='Set2',
    order=['regiao_classe_A','regiao_classe_B','regiao_classe_C'],
    ax=axs[2, 1]
)
axs[2, 1].set_title("Renda Média por Dependente")
axs[2, 1].set_xlabel("Classe")
axs[2, 1].set_ylabel("Renda Média (mil R$)")
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.3)
medianas = df_preproced.groupby('grupo_regiao')['renda_media_por_dependente'].median()
for i, grp in enumerate(['regiao_classe_A','regiao_classe_B','regiao_classe_C']):
    axs[2, 1].text(i, medianas[grp] + 3, f"Mediana: {medianas[grp]:.1f}",
                   ha='center', fontweight='bold')

# 9) Boxplot: tempo desde último serviço
sns.boxplot(
    data=df_preproced,
    x='grupo_regiao',
    y='tempo_ultimoservico',
    palette='Set2',
    order=['regiao_classe_A','regiao_classe_B','regiao_classe_C'],
    ax=axs[2, 2]
)
axs[2, 2].set_title("Tempo desde Último Serviço")
axs[2, 2].set_xlabel("Classe")
axs[2, 2].set_ylabel("Meses")
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.3)
medianas_ts = df_preproced.groupby('grupo_regiao')['tempo_ultimoservico'].median()
for i, grp in enumerate(['regiao_classe_A','regiao_classe_B','regiao_classe_C']):
    axs[2, 2].text(i, medianas_ts[grp] + 2, f"{medianas_ts[grp]:.0f}",
                   ha='center', fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
plt.rcParams['font.family'] = 'DejaVu Sans'

fig, axs = plt.subplots(ncols=2, figsize=(12, 5))

# Boxplot original
sns.boxplot(x=df_preproced['vl_salario_mil'], ax=axs[0], color="grey")
axs[0].set_title("Salário (escala normal)", fontsize=12)
axs[0].set_xlabel("Renda (mil R$)", fontsize=10)
axs[0].grid(axis='x', linestyle='--', alpha=0.3)
axs[0].tick_params(axis='y', length=0)
axs[0].spines[['top', 'right', 'left']].set_visible(False)

# Boxplot zscore
sns.boxplot(x=df_preproced['renda_zscore'], ax=axs[1], color="orange")
axs[1].set_title("Salário (escalado z-score)", fontsize=12)
axs[1].set_xlabel("Z-score", fontsize=10)
axs[1].grid(axis='x', linestyle='--', alpha=0.3)
axs[1].tick_params(axis='y', length=0)
axs[1].spines[['top', 'right', 'left']].set_visible(False)

plt.suptitle("Analisando a distribuição da renda", fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
plt.rcParams['font.family'] = 'DejaVu Sans'

fig, axs = plt.subplots(ncols=2, figsize=(12, 5))

# Boxplot zscore
sns.boxplot(x=df_preproced['renda_media_por_dependente_zscore'], ax=axs[0], color="grey")
axs[0].set_title("renda media por dependente (z-score)", fontsize=12)
axs[0].set_xlabel("z-score", fontsize=10)
axs[0].grid(axis='x', linestyle='--', alpha=0.3)
axs[0].tick_params(axis='y', length=0)
axs[0].spines[['top', 'right', 'left']].set_visible(False)

# Boxplot log1
sns.boxplot(x=df_preproced['log_renda_media_por_dependente'], ax=axs[1], color="orange")
axs[1].set_title("renda media por dependente (log1)", fontsize=12)
axs[1].set_xlabel("log1", fontsize=10)
axs[1].grid(axis='x', linestyle='--', alpha=0.3)
axs[1].tick_params(axis='y', length=0)
axs[1].spines[['top', 'right', 'left']].set_visible(False)

plt.suptitle("Analisando a renda média compartilhada por dependente", fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
colunas_desejadas = [
    'Qte_dependentes',
    'tempo_ultimoservico',
    'trabalha',
    'vl_salario_mil',
    'reg_moradia',
    'casa_propria',
    'vl_imovel_em_mil',
    'Qte_cartoes',
    'Qte_carros',
    'renda_media_por_dependente',
    'renda_media_por_dependente_zscore',
    'log_renda_media_por_dependente',
    'idade'
]


df_subset = df_preproced[colunas_desejadas]


df_subset = df_subset.apply(pd.to_numeric, errors='coerce')


matriz_correlacao = df_subset.corr()


plt.figure(figsize=(12, 8))
sns.heatmap(
    matriz_correlacao,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    linewidths=0.5,
    cbar_kws={"shrink": 0.75}
)
plt.title("Matriz de Correlação - Variáveis Selecionadas", fontsize=14, weight='bold')
plt.tight_layout()
plt.show()
